# Introduction
This notebook showcases how to use the Gemini Computer Use model to interact with a webpage, fill forms, and perform automated UI actions using screenshots and model-generated instructions.

The Gemini 2.5 Computer Use model enables building **browser control agents** that can *see* a computer screen and *interact* with it via UI actions. It is ideal for automating tasks, testing web applications, and gathering data across websites.

### **Core Concepts of Gemini Computer Use Model**

### **1. Vision + Action**

- The model is designed to **“see” the computer screen** using screenshots provided by the client.
- Along with the screenshot, the model receives the **user’s request** (e.g., “fill out this form” or “click the submit button”).
- Using this information, the model **analyzes the GUI** and decides the appropriate action to perform.
- It then generates **UI actions** as structured outputs, similar to function calls, such as:
    - `click(x, y)` → click at specific screen coordinates
    - `type("text")` → enter text in a field
    - `scroll(direction, amount)` → scroll the page

> Essentially, the model translates user intent and visual context into precise UI instructions.
> 

---

### **2. Client-Side Execution**

- The model **does not directly interact** with the browser or operating system.
- Instead, it provides **recommended actions** to your client-side application.
- The client application is responsible for:
    - Receiving the suggested UI actions.
    - Executing them in the actual environment (web browser, desktop application, etc.).
- This separation ensures that the **AI cannot accidentally perform unsafe actions**, and all execution is controlled by your application.

---

### **3. Safety System**

- Every action proposed by the model is **evaluated for risk** by an internal safety system.
- There are two main classifications:
    1. **Allowed** – The action is deemed safe and can be executed automatically without further confirmation.
    2. **Requires confirmation** – The action may have potential risk (e.g., clicking a popup or accepting a cookie banner). In this case, the client must **prompt the user** for approval before execution.

> The safety system acts as a safeguard, ensuring that the agent cannot perform unintended or potentially harmful operations without oversight.
> 

---

### **Gemini Computer Use Model Workflow**

### **1. Send Request to Model**

- The client application prepares a request to the Gemini Computer Use model.
- This request includes:
    - **The Computer Use tool** – the core module that allows the model to interpret and act on GUI screens.
    - **Custom or excluded functions** – optional instructions to restrict or extend the model’s possible actions.
    - **User input / intent** – what the user wants the agent to do (e.g., “fill in this form”).
    - **Screenshot of the current GUI** – provides visual context of the application or website the agent is interacting with.

> Essentially, you are giving the model all the context it needs to make an informed decision about what action to take.
> 

---

### **2. Receive Model Response**

- The model processes the user request and the screenshot to generate:
    - **function_call** – a structured instruction representing a UI action (e.g., `click`, `type`, `scroll`).
    - **safety_decision** (optional) – indicates whether the action is automatically safe or requires user confirmation.
- The client application receives this response and prepares to act accordingly.

---

### **3. Execute Action in Client**

- Based on the safety decision:
    1. **Allowed** – The action is safe and can be executed immediately in the client environment (browser, desktop app, etc.).
    2. **Requires confirmation** – The client must prompt the end user for approval.
        - If the user **approves**, execute the action.
        - If the user **denies**, skip execution and handle accordingly.

> This step ensures human oversight for potentially risky actions and maintains safety in automated workflows.
> 

---

### **4. Capture New Environment State**

- After executing the action (or skipping it if blocked/denied):
    - Capture a **new screenshot** showing the updated GUI state.
    - Record the **current URL** if interacting with a web application.
    - Send this back to the model as part of a **function_response**, allowing it to continue reasoning based on the latest environment.
- If the action was blocked or denied, you may:
    - Send alternative feedback to the model, or
    - End the interaction if no further actions are needed.

> This step closes the loop and provides continuous context to the model, enabling multi-step automation and decision-making.
>

In [5]:
# Setup Gemini Computer Use demo environment quietly
%pip install -q google-genai playwright
!playwright install chromium

Note: you may need to restart the kernel to use updated packages.
|                                                                                |   0% of 148.9 MiB
|■■■■■■■■                                                                        |  10% of 148.9 MiB
|■■■■■■■■■■■■■■■■                                                                |  20% of 148.9 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■                                                        |  30% of 148.9 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                                                |  40% of 148.9 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                                        |  50% of 148.9 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                                |  60% of 148.9 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                        |  70% of 148.9 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                |  80% of 148.9 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■

In [6]:
# Read .env file and set environment variables
import os
from dotenv import load_dotenv

load_dotenv(override=True)

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
print("GEMINI_API_KEY:", GEMINI_API_KEY is not None)
if not GEMINI_API_KEY:
    raise ValueError(
        "GEMINI_API_KEY environment variable not set. Please set it in the .env file."
    )

GEMINI_API_KEY: True


In [26]:
from playwright.async_api import async_playwright

pw = await async_playwright().start()
browser = await pw.chromium.launch(headless = False)
page = await browser.new_page()

await page.goto("https://scrapingbee.com/")

NotImplementedError: 